In [1]:
import re
import spacy
import pandas as pd
import emoji as emo

nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
all_stopwords = nlp.Defaults.stop_words

In [2]:
def clean_tweet(tweet, emojis=True) -> str:

# This is the noise-list
    filter_list = ["@USER",             # -> remove '@USER'
                    "CORRECTION:",      # -> remove 'CORRECTION:
                    "URL",              # -> remove URL
                    "#.+",              # -> remove hashtags
    ]

    # Clean based on regex-rules
    regex_clean_tweet = []
    for word in tweet.split():
        for item in filter_list:
            word = re.sub(f"{item}", "", word)
        if len(word) > 0:
            regex_clean_tweet.append(word)
    tweet = " ".join(regex_clean_tweet)

    if emojis:
        for word in tweet.split():
            for char in word:
                if emo.is_emoji(char):
                    emoji = char.strip()
                    tweet = tweet.replace(emoji, f' {"".join([char for char in emo.demojize(emoji) if char.isalpha()])} ')

    # Clean by lemmatizing
    tokens = nlp(tweet.lower())
    lemm_text = [word.lemma_ for word in tokens]
    no_stopwords = [word for word in lemm_text if word not in all_stopwords]
    alpha = [word for word in no_stopwords if word.isalpha()]
    clean_string = " ".join(alpha)
    
    return clean_string